In [189]:
import copy

import pymdp
from pymdp import utils
from pymdp.agent import Agent
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

In [182]:
trials = 64
steps = 2  # number of timepoints per trial

#STATES
# state if the img is cs+ or cs-
state = ['cs+', 'cs-']


#OBSERVATIONS

# morphing level binary. shock only if morph_level=6
morph_level_obs = ['1','6']
# the subject receive the electric shock?
shock_obs = ['shock', 'no shock']
# observation to discriminate if the subject is surprised or prepared. A sort of diff between user prediction and shock observation
surprised_obs = ['not_surprised','surprised']

#ACTIONS
# the subject predict if should be receive the shock or not
action = ['guess-cs+','guess-cs-']

# Define `num_states` and `num_factors` below
num_states = [len(state)] #[2]
num_factors = len(num_states) #1

#Define `num_obs` and `num_modalities` below
num_obs = [len(morph_level_obs), len(shock_obs), len(surprised_obs)] # [2,2,2]
num_modalities = len(num_obs) #3



In [178]:
def matrices(TrialEnv):
    """
    The A, B, C, D, E matrices are built.
    """
    # This is the likelihood mapping that truly describes the relationship between the
    # environment's hidden state and the observations the agent will get

    # A matrix
    # A dimension?
    A = utils.obj_array(num_modalities)
    A_morph = np.zeros((len(morph_level_obs), len(state)))
    A_shock = np.zeros((len(shock_obs), len(state)))
    A_surprise = np.zeros((len(surprised_obs), len(state)))

    #mor_lev_binary/states
    A_morph = np.array([ [0, 1],
                            [1, 0] ]
                          )

    #shock-noshock/states
    A_shock = np.array([ [0.75, 0],
                            [0.25, 1] ]
                          )

    #prepared-surprised/states
    A_surprise = np.array([ [0.5, 0.5],
                            [0.5, 0.5] ]
                          )
    A[0] = A_morph
    A[1] = A_shock
    A[2] = A_surprise
    # B matrix
    # The transition mapping that truly describes the environment's dynamics

    #B as an identity matrix. no state transitions
    B = utils.obj_array(num_factors)
    B_ = np.zeros((num_states[0], num_states[0], len(action)))

    B_[:,:,0] = np.eye(num_states[0])
    B_[:,:,1] = np.eye(num_states[0])

    '''B_[0, :,:] = np.array([ [1, 0],
                           [0, 1] ])
    B_[1, :,:] = np.array([ [1, 0],
                       [0, 1] ])'''

    B[0]=B_
    B_gm = copy.deepcopy(B)  # make a copy of the true transition likelihood to initialize the transition model
    #???
    #A_gm = copy.deepcopy(A_gp)  # make a copy of the true observation likelihood to initialize the observation model
    #???
    #B_gm = copy.deepcopy(B_gp)  # make a copy of the true transition likelihood to initialize the transition model


    # This is the matrix representing the preferences over outcomes
    #prepared-surprised/cs+-
    C = utils.obj_array_zeros(num_obs)          #num modalities
    C[0] = np.array([4.0,-6.0])
    C[1] = np.zeros(2)
    C[2] = np.zeros(2)


    return A, B_gm, C

In [186]:
class TrialEnv(object):
    def __init__(self,data):
        """
        Constructor of the environment
        :param data: Dataframe with 4 columns: 'trial_number','morphing_level','condition','shock'

        'trial_number': number of trial in the experiment
        'morphing_level': 1 if cs+, 0 otherwise
        'condition': 1 cs+, 0 otherwise
        'shock': 1 if the shock is given at this trial, 0 otherwise

        """
        self.data = data

    def get_observation(self,param,trial_number):
        '''

        :param param: column of the dataframe that you want to read
        :param trial_number:
        :return: value of the specific column in the trial_number row
        '''
        return self.data.iloc(trial_number)[param]


    def step(self, action,current_trial):
        '''
        Take a step in the environment given an action

        :param action:
        :param current_trial:
        :return:
        '''
        surprised_obs = None

        if action == 'guess cs+':
            shock_obs = self.get_observation(param='shock',trial_number=current_trial)

            if shock_obs >0.5: # in this case shock and i have predicted cs+ #TODO change this
                surprised_obs = self.get_surprised_low()

            else: # in this case no shock and i have predicted cs+
                surprised_obs = self.get_surprised_high()


        if action == 'guess cs-':
            shock_obs = self.get_observation(param='shock',trial_number=current_trial)

            if shock_obs >0.5: # in this case shock and prediction cs- #TODO change this
                surprised_obs = self.get_surprised_high()

            else: # in this case no shock and prediction cs-
                surprised_obs = self.get_surprised_low()

        if surprised_obs is None:
            print('ERROR: Action not supported!')

        return surprised_obs


    def get_surprised_high(self):
        return 1

    def get_surprised_low(self):
        return 0


In [179]:
A,B,C = matrices(TrialEnv)

In [195]:
import os
os.chdir('..')

In [206]:
def load_data(columns=[], binary_morphing_level=False, shock_as_int = False,subject_number=0):
    df = pd.read_csv('./data/newLookAtMe/newLookAtMe01.csv')
    df = df.filter(columns)
    print(df)
    if binary_morphing_level:
        df = df
    if shock_as_int:
        df = df

    return df

In [207]:

data= load_data(columns=['index','morphing level','shock'] ,binary_morphing_level=True, shock_as_int = True)

     morphing level  shock
0                 1  False
1                 6  False
2                 6  False
3                 1  False
4                 1  False
..              ...    ...
155               4  False
156               5  False
157               1  False
158               1  False
159               6  False

[160 rows x 2 columns]


In [188]:
# define an agent
my_agent = Agent(A = A, B = B, C = C)

# define an environment
my_env = TrialEnv(data= data)


initial_action = 'first_action'

# set an initial action
action = initial_action

# iterate over all trials
for trial in range (trials) :

    for step in range (steps) : #each trial divided in two steps

        stimul_obs = my_env.get_observation(param='stimulus',trial_number=trial)

        if step == 0:
            surpr_obs = 'null'
            shock_obs = 'null'

        if step == 1:
            shock_obs = my_env.get_observation(param='shock',trial_number=trial)

        my_agent.infer_states (observation=[stimul_obs,shock_obs,surpr_obs]) # agent update beliefs about hidden states given observations

        my_agent.infer_policies () # inferring policies and sampling actions from the posterior

        action = my_agent.sample_action()

        if step==0:
            surpr_obs = my_env.step(action)



AttributeError: 'int' object has no attribute 'iloc'

In [169]:
len(C)

3

In [142]:
B.sum(axis=0)

array([[[1., 0.],
        [0., 1.]],

       [[1., 0.],
        [0., 1.]]])

In [141]:
B[0][1].sum(axis =0)

array([1., 1.])

In [117]:
B[0].sum(axis = 0)

array([[1., 0.],
       [0., 1.]])

In [71]:
B_locs = np.eye(2).reshape(2,2,1)
B_locs

array([[[1.],
        [0.]],

       [[0.],
        [1.]]])

In [72]:
B_locs = np.tile(B_locs, (1, 1, 2))

In [73]:
B_locs

array([[[1., 1.],
        [0., 0.]],

       [[0., 0.],
        [1., 1.]]])

In [75]:
B_locs.shape

(2, 2, 2)

In [76]:
 B_locs = B_locs.transpose(1, 2, 0)

In [77]:
B_locs

array([[[1., 0.],
        [1., 0.]],

       [[0., 1.],
        [0., 1.]]])

In [130]:
B_location = np.zeros((4, 3, 4))
B_location

array([[[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]],

       [[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]],

       [[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]],

       [[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]]])

In [131]:
B_location[:, :, 0] = np.array([[1, 0, 0, 1],
                                [0, 0, 1, 0],
                                [0, 0, 0, 0]])
B_location
'''B_location[:, :, 1] = np.array([[0, 0, 0, 0],
                                [0, 0, 1, 0],
                                [0, 0, 0, 0]])
B_location[:, :, 2] = np.array([[0, 0, 0, 0],
                                [1, 0, 1, 1],
                                [0, 0, 0, 0]])
B_location[:, :, 3] = np.array([[0, 0, 0, 0],
                                [0, 0, 1, 0],
                                [1, 0, 0, 1]])'''

ValueError: could not broadcast input array from shape (3,4) into shape (4,3)

In [129]:
B_location

array([[[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]],

       [[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]],

       [[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]],

       [[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]]])

In [145]:
num_states

[2]

In [158]:
B = utils.obj_array(num_factors)
B_ = np.zeros((num_states[0], num_states[0], len(action)))

B_[:,:,0] = np.eye(num_states[0])
B_[:,:,1] = np.eye(num_states[0])

In [159]:
B_

array([[[1., 1.],
        [0., 0.]],

       [[0., 0.],
        [1., 1.]]])

In [175]:
prova=np.zeros((3, 2, 4))
prova[:,:,0]


array([[0., 0.],
       [0., 0.],
       [0., 0.]])

In [177]:
shape = ((len(morph_level_obs), len(state)))
prova = np.arange(len(morph_level_obs)*len(state)).reshape(shape)
prova

array([[0, 1],
       [2, 3]])